In [8]:
###################################### Inter Bearer Token ################################################################################
bearer_token = ""

In [9]:

with open('Query_File.txt', 'r') as file:
    Query = file.readlines()


In [10]:
###################################### Inter Number of Pack Tweets ################################################################################

Number_of_Pack_Tweets = 1

In [11]:
import tweepy
import time
import pandas as pd
import csv
import sys
import numpy as np
import os
from datetime import datetime, timezone


In [12]:
local_time = datetime.now(timezone.utc).astimezone()
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [13]:
def Collect_Tweet(Query, End_time, query_counter):


    hoax_tweets = []
    for response in tweepy.Paginator(client.search_all_tweets, 
                                    query = Query+' -is:retweet lang:en',
                                    user_fields = ['username', 'public_metrics', 'description', 'location'],
                                    tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                    end_time = End_time,
                                    start_time = '2007-12-17T15:25:23+00:00',
                                    expansions = 'author_id',
                                    max_results=500, limit=320):
        time.sleep(1)
        hoax_tweets.append(response)
        result = []
        user_dict = {}
        # Loop through each response object
        for response in hoax_tweets:
            # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
            for user in response.includes['users']:
                user_dict[user.id] = {'username': user.username, 
                                    'followers': user.public_metrics['followers_count'],
                                    'tweets': user.public_metrics['tweet_count'],
                                    'description': user.description,
                                    'location': user.location
                                    }
            for tweet in response.data:
                # For each tweet, find the author's information
                author_info = user_dict[tweet.author_id]
                # Put all of the information we want to keep in a single dictionary for each tweet
                result.append({'author_id': tweet.author_id, 
                            'username': author_info['username'],
                            'author_followers': author_info['followers'],
                            'author_tweets': author_info['tweets'],
                            'author_description': author_info['description'],
                            'author_location': author_info['location'],
                            'text': tweet.text,
                            'created_at': tweet.created_at,
                            'retweets': tweet.public_metrics['retweet_count'],
                            'replies': tweet.public_metrics['reply_count'],
                            'likes': tweet.public_metrics['like_count'],
                            'quote_count': tweet.public_metrics['quote_count']
                            })


        df = pd.DataFrame(result)
        df.to_csv(str(query_counter+1)+'.csv')
                                                                                                                                                                                                    

In [ ]:
for iii in range (Number_of_Pack_Tweets):

    for query_counter in range(len(Query)):
        if os.path.exists(Query[query_counter]+'.csv'):
            df = pd.read_csv(Query[query_counter]+'.csv', encoding="ISO-8859-1")
            created_at = pd.to_datetime(df['created_at'], utc=True)
            End_time = created_at.sort_values().min().isoformat()
        else:
            End_time = local_time.isoformat()

        time.sleep(10)    
        Collect_Tweet(Query[query_counter],End_time,query_counter)
   